# **Семинар 2. Обнаружение дрифта данных и модели с помощью Evidently**

---

## **Цель семинара**

Научиться анализировать и визуализировать изменения в данных и поведении ML-модели с помощью библиотеки **Evidently AI**.

После занятия студент сможет:

- определять дрифт данных (data drift) и дрифт модели (target drift);  
- строить интерактивные отчёты о дрифте;  
- интерпретировать статистические показатели (PSI, KS, p-value);  
- встраивать Evidently в пайплайн CI/CD для мониторинга моделей.

---


In [ ]:
# 1. Установка библиотек и импорт зависимостей

!pip install evidently pandas scikit-learn matplotlib

## **2. Подготовка данных**

Создадим два датафрейма:  
- `reference_data` — исторические (тренировочные) данные;  
- `current_data` — новые данные из продакшена.  

На практике дрифт возникает, если распределение признаков во «входящих» данных начинает отличаться от обучающего.


In [ ]:
import pandas as pd
from sklearn.datasets import load_iris
import numpy as np

# Загружаем базовый датасет
iris = load_iris(as_frame=True)
ref_df = iris.frame.copy()

# Создаём "текущие" данные с дрифтом
cur_df = ref_df.copy()
cur_df["sepal length (cm)"] = cur_df["sepal length (cm)"] * np.random.uniform(0.8, 1.2, len(cur_df))
cur_df["petal width (cm)"] = cur_df["petal width (cm)"] + np.random.normal(0, 0.3, len(cur_df))

# Добавляем небольшой сдвиг в распределении target
cur_df["target"] = cur_df["target"].apply(lambda x: 0 if np.random.rand() < 0.4 else x)

ref_df.head(10)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
5,5.4,3.9,1.7,0.4,0
6,4.6,3.4,1.4,0.3,0
7,5.0,3.4,1.5,0.2,0
8,4.4,2.9,1.4,0.2,0
9,4.9,3.1,1.5,0.1,0


## **3. Создание отчёта с Evidently**

Evidently позволяет визуализировать изменения распределений признаков и целей между двумя выборками.


In [ ]:
pip install evidently==0.3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.0 MB/s eta 0:00:00
Reason for being yanked: The package includes unnecessary dependency on tkinter
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 87.8 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.3
    Uninstalling pydantic-2.12.3:
      Successfully uninstalled pydantic-2.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.53.0 requires pydantic<3.0.0,>=2.9.0, but you have pydantic 1.10.24 which is incompatible.
pydantic-settings 2.12.0 requires pydantic>=2.7.0, but you have pydantic 1.10.24 which is incompatible.
mcp 1.23.1 requires pydantic<3.0.0,>=2.11.0, but you have pydantic 1.10.24 which is incompatible.
google-adk 1.20.0 requires pydantic<3.0

In [ ]:
from evidently import Report

In [ ]:
from evidently import metrics

In [ ]:
from evidently.metrics

SyntaxError: invalid syntax (ipython-input-4033987833.py, line 1)

In [ ]:
from evidently import Report
from evidently.metric_preset import DataDriftPreset
from evidently.metrics import TargetDriftPreset

report = Report(metrics=[
    DataDriftPreset(),
    TargetDriftPreset()
])

report.run(reference_data=ref_df, current_data=cur_df)
report.save_html("drift_report.html")


ModuleNotFoundError: No module named 'evidently.metric_preset'

После выполнения появится файл **`drift_report.html`**,  
который можно открыть в браузере и увидеть:
- гистограммы изменений признаков;  
- p-value для каждого признака;  
- общий индекс дрифта по датасету.  


## **4. Интерпретация отчёта**

Обратите внимание на ключевые показатели:

| **Метрика** | **Описание** |
|--------------|--------------|
| **PSI (Population Stability Index)** | Измеряет изменение распределения признака между reference и current. Значения > 0.25 — сильный дрифт. |
| **KS (Kolmogorov–Smirnov)** | Тест на равенство распределений. p-value < 0.05 → признак изменился. |
| **Share of drifted features** | Доля признаков, у которых зафиксирован дрифт. |

Если доля «дрифтовых» признаков > 30 %, модель, вероятно, нуждается в переобучении.


In [ ]:
X - reference data
Y - current data

x = [1, 2, 3]

I(X_i <= x)

F(2) = 2/3

Y = [2, 4, 6]

In [ ]:
PSI_i = (A_i-B_i)*ln(A_i/B_i)
PSI = sum(PSI_i)

In [ ]:
import numpy as np

def psi_score (reference, current, n_bins=10, eps=1e-6):
  ref = np.asarray(reference).astype(float)
  cur = np.asarray(current).astype(float)

  ref = ref[~np.isnan(ref)]
  cur = cur[~np.isnan(cur)]

  bin_edges = np.histogram_bin_edges(ref, bins=n_bins)

  ref_counts = np.histogram(ref, bins=bin_edges)[0]
  cur_counts = np.histogram(cur, bins=bin_edges)[0]

  ref_perc = ref_counts / ref_counts.sum()
  cur_perc = cur_counts / cur_counts.sum()

  ref_perc = np.clip(ref_perc, eps,1.0)
  cur_perc = np.clip(cur_perc, eps,1.0)

  psi_components = (ref_perc - cur_perc) * np.log(ref_perc / cur_perc)
  total_psi = psi_components.sum()

  bin_table = {
      "bin_edges": bin_edges,
      "ref_counts": ref_counts,
      "cur_counts": cur_counts,
      "ref_perc": ref_perc,
      "cur_perc": cur_perc,
      "psi_components": psi_components
  }

  return float(total_psi), bin_table

In [ ]:
ref = np.random.normal(loc=0, scale=1, size=1000)
cur = np.random.normal(loc=0, scale=1, size=1000)


psi_score, bin_table = psi_score(ref, cur)
print('PSI: ', psi_score)
print('Доля дрейф-компонент', bin_table["psi_components"])

PSI:  0.2847230870999765
Доля дрейф-компонент [0.01744435 0.02300257 0.06754765 0.0368301  0.00702613 0.00063988
 0.00966883 0.0404098  0.07393001 0.00822377]


In [ ]:
import numpy as np

def psi_score (reference, current, n_bins=10, eps=1e-6):
  ref = np.asarray(reference).astype(float)
  cur = np.asarray(current).astype(float)

  ref = ref[~np.isnan(ref)]
  cur = cur[~np.isnan(cur)]

  bin_edges = np.histogram_bin_edges(ref, bins=n_bins)

  ref_counts = np.histogram(ref, bins=bin_edges)[0]
  cur_counts = np.histogram(cur, bins=bin_edges)[0]

  ref_perc = ref_counts / ref_counts.sum()
  cur_perc = cur_counts / cur_counts.sum()

  ref_perc = np.clip(ref_perc, eps,1.0)
  cur_perc = np.clip(cur_perc, eps,1.0)

  psi_components = (ref_perc - cur_perc) * np.log(ref_perc / cur_perc)
  total_psi = psi_components.sum()

  bin_table = {
      "bin_edges": bin_edges,
      "ref_counts": ref_counts,
      "cur_counts": cur_counts,
      "ref_perc": ref_perc,
      "cur_perc": cur_perc,
      "psi_components": psi_components
  }

  return float(total_psi), bin_table


ref = np.random.normal(loc=0, scale=1, size=1000)
cur = np.random.normal(loc=0, scale=2, size=1000)

psi_score, bin_table = psi_score(ref, cur)
print('PSI: ', psi_score)
print('Доля дрейф-компонент', bin_table["psi_components"])

PSI:  0.48129330642664775
Доля дрейф-компонент [0.10969962 0.02056569 0.00856144 0.01084707 0.07500538 0.02340286
 0.00761137 0.00320921 0.10743926 0.11495141]


In [ ]:
def ecdf(values, grid):
  values = np.sort(np.asarray(values))
  return np.searchsorted(values, grid) / values.sizw

def ks_score(reference, current):
  ref = np.asarray(reference).astype(float)
  cur = np.asarray(current).astype(float)

  ref = ref[~np.isnan(ref)]
  cur = cur[~np.isnan(cur)]

  grid = np.sort(np.unique(np.concatenate([ref, cur])))

  F = ecdf(ref, grid)
  G = ecdf(cur, grid)

  diff = np.abs(F-G)
  D = diff[int(np.argmax(diff))]
  return D, grid, F, G, int(np.argmax(diff))

## **5. Добавление автоматического порога для оповещения**


In [ ]:
drift_dict = report.as_dict()

# Проверяем общий статус дрифта по датасету
if drift_dict["metrics"][0]["result"]["dataset_drift"]:
    print("⚠️ Drift detected! Trigger retraining.")
else:
    print("✅ No significant drift detected.")


## еще примеры

In [ ]:
import pandas as pd
import numpy as np
from evidently import Report
from evidently.metric_presets import DataDriftPreset

ref = pd.DataFrame({
    "city": np.random.choice(["Moscow", "SPB", "Kazan"], size=500)
})

cur = pd.DataFrame({
    "city": np.random.choice(["Moscow", "SPB", "Novosibirsk"], size=500)
})

report = Report(metrics=[DataDriftPreset()])
report.run(reference_data=ref, current_data=cur)
report


ModuleNotFoundError: No module named 'evidently.metric_presets'

In [ ]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

data = load_iris()
df = pd.DataFrame(data.data, columns=data.feature_names)

ref_df = df.copy()
cur_df = df.copy()

cur_df["sepal width (cm)"] *= 1.25  # drift только одного признака

report = Report(metrics=[DataDriftPreset()])
report.run(reference_data=ref_df, current_data=cur_df)
report


In [ ]:
ref = pd.DataFrame({
    "sales": np.random.normal(loc=100, scale=10, size=500),
    "month": np.random.choice(["Jan", "Feb", "Mar"], size=500)
})

cur = pd.DataFrame({
    "sales": np.random.normal(loc=150, scale=15, size=500),
    "month": np.random.choice(["Jun", "Jul", "Aug"], size=500)
})

report = Report(metrics=[DataDriftPreset()])
report.run(reference_data=ref, current_data=cur)
report


In [ ]:
ref_df = pd.DataFrame({
    "x": np.random.normal(0, 1, 400),
    "target": np.random.choice([0,1], 400, p=[0.8, 0.2])
})

cur_df = pd.DataFrame({
    "x": np.random.normal(1, 1.5, 400),
    "target": np.random.choice([0,1], 400, p=[0.5, 0.5])
})

from evidently.metric_preset import DataDriftPreset, TargetDriftPreset

report = Report(metrics=[DataDriftPreset(), TargetDriftPreset()])
report.run(reference_data=ref_df, current_data=cur_df)
report


In [ ]:
ref = pd.DataFrame({
    "age": np.random.normal(40, 5, 400),
    "income": np.random.normal(3000, 500, 400)
})

cur = pd.DataFrame({
    "age": np.random.normal(25, 6, 400),
    "income": np.random.normal(1200, 300, 400)
})

report = Report(metrics=[DataDriftPreset()])
report.run(reference_data=ref, current_data=cur)
report


## **6. Интерактив: когда дрифт не является проблемой?**

- **Сезонность:** спрос на услуги растёт летом — дрифт естественный.  
- **Промоакции:** временное изменение распределения клиентов.  
- **Новый продукт:** структура данных изменилась, но модель адаптирована.

 **Вывод:**  
не каждый дрифт = деградация.  
важно анализировать контекст и бизнес-метрики.


## **7. Интеграция Evidently в CI/CD**

Пример простого шага проверки в GitHub Actions:

```yaml
- name: Check for data drift
  run: |
    python detect_drift.py



---

```markdown
##**Мини-практикум**

1. Измените только один признак и повторите анализ.  
2. Сравните отчёт и определите, какие метрики реагируют быстрее.  
3. Добавьте метрику `TargetDriftPreset()` и оцените, изменилась ли целевая переменная.  
4. Сохраните HTML-отчёт и прикрепите к отчёту в GitHub.


## **9. Итог семинара**

После занятия вы получили:

- отчёт о дрифте данных и целей;  
- навыки интерпретации статистических показателей;  
- шаблон автоматического детектора дрифта для CI/CD;  
- понимание, когда и зачем перезапускать обучение модели.

**Мониторинг дрифта — неотъемлемая часть MLOps:**
он позволяет поддерживать актуальность модели и предотвращать её деградацию в продакшене.
